# 智谱大模型

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 智谱API

### 同步调用

In [340]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
        {"role": "user", "content": "讲一个关于程序员的笑话"},
    ],
)

print(response.choices[0].message)

content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。' role='assistant' tool_calls=None


### 异步调用

<div class="alert-info">
    <b>注意:</b><br>
    智谱API的异步调用不支持流。
</div>

In [ ]:
import time
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.asyncCompletions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {
            "role": "user",
            "content": "请你作为童话故事大王，写一篇短篇童话故事，故事的主题是要永远保持一颗善良的心，要能够激发儿童的学习兴趣和想象力，同时也能够帮助儿童更好地理解和接受故事中所蕴含的道理和价值观。"
        }
    ],
)

task_id = response.id
prev_status = ''
get_cnt = 0

while prev_status != 'SUCCESS' and task_status != 'FAILED' and get_cnt <= 40:
    result_response = client.chat.asyncCompletions.retrieve_completion_result(id=task_id)
    current_status = result_response.task_status
    if(current_status == "PROCESSING"):
        print(".", end="", flush=True)
    elif(current_status == "SUCCESS"):
        print()
        print(result_response.choices[0].message)
    prev_status = current_status

    time.sleep(2)
    get_cnt += 1

### 事件流

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",  # 填写需要调用的模型名称
    messages=[
        {"role": "system", "content": "你是一个乐于解答各种问题的助手，你的任务是为用户提供专业、准确、有见地的建议。"},
        {"role": "user", "content": "我对太阳系的行星非常感兴趣，特别是土星。请提供关于土星的基本信息，包括其大小、组成、环系统和任何独特的天文现象。"},
    ],
    stream=True,
)
for chunk in response:
    # print(chunk)
    print("-" * 80)
    print(chunk.choices[0].delta.content, end="", flush=True)

### 函数回调

In [14]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",
    messages = [
        {
            "role": "user",
            "content": "你能帮我查询2024年1月1日从北京南站到上海的火车票吗？"
        }
    ],
    tools = [
        {
            "type": "function",
            "function": {
                "name": "query_train_info",
                "description": "根据用户提供的信息，查询对应的车次",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "departure": {
                            "type": "string",
                            "description": "出发城市或车站",
                        },
                        "destination": {
                            "type": "string",
                            "description": "目的地城市或车站",
                        },
                        "date": {
                            "type": "string",
                            "description": "要查询的车次日期",
                        },
                    },
                    "required": ["departure", "destination", "date"],
                },
            }
        }
    ],
    tool_choice="auto",
)
print(response.choices[0].message)

content=None role='assistant' tool_calls=[CompletionMessageToolCall(id='call_8367745282338832529', function=Function(arguments='{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', name='query_train_info'), type='function')]


## 作为 RunnableLambda

### 基本功能

In [97]:
from zhipuai import ZhipuAI

client = ZhipuAI()

def zhipu_chat(messages: [str]):
    response = client.chat.completions.create(
        model="glm-3-turbo",
        messages=messages,
    )
    return(response)

In [98]:
prompt = [
    {"role": "user", "content": "讲一个关于程序员的笑话"},
]

In [99]:
zhipu_chat(prompt)

Completion(model='glm-3-turbo', created=1708146156, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367745522857040954', id='8367745522857040954', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

### 包装为 Runnable

In [100]:
from langchain_core.runnables import RunnableLambda

my_zhipu_chat = RunnableLambda(zhipu_chat)

In [101]:
my_zhipu_chat.invoke(prompt)

Completion(model='glm-3-turbo', created=1708146224, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367743907949124838', id='8367743907949124838', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

## ZhipuAIChatTiny：最小实现

### 定义

In [107]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from typing import Any, Dict, Iterator, List, Optional, cast

In [161]:
class ZhipuAIChatTiny(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None
    
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        try:
            # 声明 ZhipuAI 的客户端
            from zhipuai import ZhipuAI
            self.client = ZhipuAI()
        except ImportError:
            raise RuntimeError(
                "Could not import zhipuai package. "
                "Please install it via 'pip install zhipuai'"
            )

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
        )

        content = response.choices[0].message.content

        return ChatResult(
            generations=[ChatGeneration(message=AIMessage(content=content))]
        )

### 使用 LLM

In [162]:
llm_tiny = ZhipuAIChatTiny()

In [163]:
llm_tiny.invoke("讲一个关于程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人叫做"裸奔者"。')

### 使用 LLM + OutputParser

In [164]:
chain = llm_tiny | StrOutputParser()
chain.invoke("讲一个关于程序员的一句话笑话")

'为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。'

### 使用 Prompt + LLM + OutputParser

In [165]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("讲一个关于{topic}的笑话")
chain = prompt | llm_tiny | StrOutputParser()

chain.invoke({"topic": "程序员"})

'好的，下面是一个关于程序员的笑话：\n\n为什么程序员不喜欢自然界？\n\n因为那里有太多的bugs（虫子/错误）！\n\n希望这个笑话能让您开心一下！'

### 使用流式输出

<div class="alert-warning", style="padding: 5px">
    <b>注意：</b><br>
    因为我们没有实现 stream 方法，基类中的实现将自动调用 invoke 方法，然后一次性返回结果。
</div>

In [166]:
for chunk in llm_tiny.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么程序员总是混淆圣诞节和万圣节？因为 Oct 31 等于 Dec 25。_

## ZhipuAIChatStream：支持流

### 定义

In [167]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

from typing import Any, Dict, Iterator, List, Optional, cast
from langchain_core.pydantic_v1 import BaseModel, Field

In [269]:
class ZhipuAIChatStream(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        try:
            # 声明 ZhipuAI 的客户端
            from zhipuai import ZhipuAI
            self.client = ZhipuAI()
        except ImportError:
            raise RuntimeError(
                "Could not import zhipuai package. "
                "Please install it via 'pip install zhipuai'"
            )

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=False
        )

        choice = response.choices[0]

        return ChatResult(
            generations=[
                ChatGeneration(
                    message=AIMessage(content=choice.message.content),
            )],
        )

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        """使用 ZhiputAI 的事件流用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        # 使用流输出
        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=True
        )

        for chunk in response:
            choice = chunk.choices[0]
            yield ChatGenerationChunk(
                message=AIMessageChunk(content=choice.delta.content),
            )

### 使用 LLM

In [270]:
llm_stream = ZhipuAIChatStream()

In [271]:
llm_stream.invoke("讲一个程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。')

### 使用流式输出

In [272]:
for chunk in llm_stream.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么_程序_员_总是_混淆_圣诞_节_和_万_圣_节_？_因为_ Oct_ _3_1_ _等于_ Dec_ _2_5_。__

<div class="alert-success", style="padding: 5px">
    <b>成功了！</b><br>
    现在运行上面的代码，应当可以看到流式输出！
</div>

## ZhipuAIChat：完整实现

完整的实现较为繁琐，可以在前面实践的基础上补充：

- 支持所有模型参数
- 支持异步方法
- 支持事件流推送
- 支持智谱的Tool回调
- 支持内置的search工具
- 支持内置的检索工具
- 支持图片生成能力
- 支持调用中的异常
- 提供便利的bind_tools方法
- 提供基于Tool调用的Agent
- ...


In [ ]:
!pip install -U langchain_chinese

<div>
    <b>ZhipuAIChat的完整实现已经发布</b><br>
    我将完整的实现作为 langchain_chinese 包的一部份发布了。
</div>

**项目地址** [https://pypi.org/project/langchain_chinese/](https://pypi.org/project/langchain_chinese/)<br> 
**源代码地址** [https://github.com/arcstep/langchain_chinese](https://github.com/arcstep/langchain_chinese)

你可以通过 pip 安装：

```
pip install langchain_chinese
```

In [ ]:
!poetry add langchain_chinese

In [336]:
from langchain_chinese import ZhipuAIChat

In [337]:
llm = ZhipuAIChat()

In [338]:
llm.invoke("讲个笑话来听吧")

AIMessage(content='好的，我来讲一个轻松的笑话给您听。\n\n有一天，一只乌龟走进了一个酒吧，跟酒保说：“请给我来杯热腾腾的牛肉汤。”\n\n酒保有些惊讶地看着乌龟说：“你确定你要牛肉汤？但是我们这里是酒吧啊。”\n\n乌龟回答：“我知道，但是这么冷的天气，再喝酒我怕冻成龟汤了。”')

In [339]:
for s in llm.stream("我父母结婚为什么不邀请我参加？"):
    print(s.content, end="|", flush=True)

当|面对|这个问题|时|，|我们可以|根据|两种|不同的|情况进行|分析|：|

-| 你|还没|出生|：|在|大多数|情况下|，|父母|在|结婚|时|，|他们的|孩子|还未|出生|。|也就是说|，|你|之所以|没有被|邀请|，|是因为|你还|未来|到|这个|世界上|。|这是|最|常见|和|最|普遍|的情况|。|
-| 你|已|出生|但是|年|幼|：|如果你|在你的|父母|结婚|时|已经|出生|了|，|那么|你可能|没有被|邀请|是因为|你|太小|，|无法|理解|或|参与|婚礼|的全部|流程|。|在|许多|文化|中|，|小孩|往往|在|一定|年龄|后|才会|开始|参加|一些|正式|的|社交|活动|，|包括|婚礼|。|或者|有时候|父母|可能|认为|儿童|在|婚礼|上|可能会|感到|无聊|或|累|。|

总的来说|，|请|记住|，|这不是|你的|错|，|也|并不|表示|你的|父母|不|爱你|。|他们|有|他们的|理由|，|这|可能|只是|因为|某些|实际情况|或者|社会|习俗|。|若|这是|出于|你的|好奇心|和对|父母|婚礼|典礼|的|渴望|，|你可以|尝试|跟|他们|聊聊|，|看看|他们|是否|愿意|分享|当时的|照片|、|故事|或|录像|。|这|无疑是|更|了解|他们|爱情|故事|的好|方法|。||

In [341]:
abc = ZhipuAIChat(model="ABC")
abc

ZhipuAIChat(client=<zhipuai._client.ZhipuAI object at 0x1192c81c0>)

In [342]:
abc.model

'glm-3-turbo'